In [121]:
#!/usr/bin/sage

import sys
print("I caught an anglerfish in the sea! ")
sys.stdout.flush()

from hashlib import sha256
from Crypto.Util.number import bytes_to_long
from random import SystemRandom
import ast

n = 100
m = 100
q = 5
FF.<x> = GF(q)

def apply(F, v):
    out = []
    for i in range(m):
        out.append((v.T * F[i] * v)[0, 0])
    return matrix(FF, m, 1, out)

def apply_verif_info(F, a, b):
    out = []
    for i in range(m):
        out.append((a.T * (F[i] + F[i].T) * b)[0, 0])
    return matrix(FF, m, 1, out)

def create_pok(v, s, F):
    proofs = []
    for i in range(64):
        t = matrix(FF, n, 1, [FF.random_element() for i in range(n)])
        com = apply(F, t)
        verif = apply_verif_info(F, t, s)
        a = list(FF)[sha256(bytes([list(FF).index(i[0]) for i in list(com) + list(v) + list(verif)])).digest()[0] % len(list(FF))]
        proofs.append((com, t - a * s, verif))
    return proofs

def verif_pok(v, F, pis):
    coms = []
    for pi in pis:
        com = pi[0]
        assert com not in coms
        coms.append(com)
        resp = pi[1]
        verif = pi[2]
        a = list(FF)[sha256(bytes([list(FF).index(i[0]) for i in list(com) + list(v) + list(verif)])).digest()[0] % len(list(FF))]
        out1 = apply(F, resp)
        out2 = com + (a * a) * v - a * verif
        assert out1 == out2

I caught an anglerfish in the sea! 


In [2]:
from pwn import *

In [ ]:
r =remote("be.ax",31106)

In [146]:
print(r.recvline())
t1=r.recvuntil("C")[:-2]

b'I caught an anglerfish in the sea! \n'


/tmp/ipykernel_6839/3180039595.py:2: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  t1=r.recvuntil("C")[:-Integer(2)]


In [147]:
t2=t1.decode().split("\n")

In [148]:
t2 = [eval(i[4:]) for i in t2]

In [149]:
r.recvuntil("=")

/tmp/ipykernel_6839/2618346645.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil("=")


b'an you catch an anglerfish? \nseed ='

In [150]:
gen_seed=eval(r.recvuntil("v")[:-2])

/tmp/ipykernel_6839/740789318.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  gen_seed=eval(r.recvuntil("v")[:-Integer(2)])


In [151]:
gen_seed = bytes(gen_seed)
F = []
for i in range(m):
    cur = []
    for j in range(n):
        cur.append([])
        for k in range(n):
            cur[-1].append(list(FF)[sha256(gen_seed).digest()[0] % len(list(FF))])
            gen_seed = sha256(gen_seed).digest()
    F.append(matrix(FF, n, n, cur))

In [152]:
r.recv()

b' = [0, 0, 2, 4, 2, 2, 0, 0, 3, 3, 3, 4, 4, 3, 0, 2, 3, 1, 3, 1, 4, 4, 2, 1, 0, 1, 1, 0, 3, 1, 4, 4, 2, 4, 2, 2, 4, 0, 3, 2, 3, 0, 0, 1, 3, 1, 0, 1, 0, 0, 1, 3, 2, 1, 3, 4, 1, 3, 1, 3, 4, 3, 2, 0, 1, 0, 3, 2, 4, 4, 3, 2, 0, 0, 1, 2, 3, 4, 3, 1, 2, 1, 2, 0, 1, 1, 1, 2, 3, 1, 2, 1, 4, 3, 0, 2, 0, 3, 1, 2]\nm0 = '

In [ ]:
r.recvuntil("=")
v=eval(r.recvline()[:-1])

/tmp/ipykernel_6839/358877357.py:11: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil("=")


messed up taking v so filled manually

In [153]:
v= [0, 0, 2, 4, 2, 2, 0, 0, 3, 3, 3, 4, 4, 3, 0, 2, 3, 1, 3, 1, 4, 4, 2, 1, 0, 1, 1, 0, 3, 1, 4, 4, 2, 4, 2, 2, 4, 0, 3, 2, 3, 0, 0, 1, 3, 1, 0, 1, 0, 0, 1, 3, 2, 1, 3, 4, 1, 3, 1, 3, 4, 3, 2, 0, 1, 0, 3, 2, 4, 4, 3, 2, 0, 0, 1, 2, 3, 4, 3, 1, 2, 1, 2, 0, 1, 1, 1, 2, 3, 1, 2, 1, 4, 3, 0, 2, 0, 3, 1, 2]

In [72]:
def find():
 a=1
 while a:
    resp,verif = matrix(FF, n, 1, [FF.random_element() for i in range(n)]),matrix(FF, n, 1, [FF.random_element() for i in range(n)])
    com = apply(F,resp)
    a = list(FF)[sha256(bytes([list(FF).index(i[0]) for i in list(com) + list(v) + list(verif)])).digest()[0] % len(list(FF))]
 return com,resp,verif

In [154]:
v = matrix(FF, m, 1, [list(FF)[i] for i in v])

In [162]:
pok = find()

Testing taking input so had to do 2 iterations manually

In [167]:
r.sendline(f'{pok[2].list()}')

/tmp/ipykernel_6839/3488409110.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(f'{pok[Integer(2)].list()}')


In [166]:
r.recv()

b'm2 = '

In [168]:
for i in range(62):
    pok = find()
    r.recv()
    r.sendline(f'{pok[0].list()}')
    r.recv()
    r.sendline(f'{pok[1].list()}')
    r.recv()
    r.sendline(f'{pok[2].list()}')
print(r.recvline())

0


/tmp/ipykernel_6839/3282014646.py:5: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(f'{pok[Integer(0)].list()}')
/tmp/ipykernel_6839/3282014646.py:7: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(f'{pok[Integer(1)].list()}')
/tmp/ipykernel_6839/3282014646.py:9: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(f'{pok[Integer(2)].list()}')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
b'corctf{extra_equations_means_extra_information_isnt_zk}\n'
